## Imports

In [11]:
from datasets import Dataset,load_dataset
import json
import pandas as pd

## Load Datasets

In [2]:
merve = load_dataset("merve/poetry")

Using custom data configuration merve--poetry-f27664155e5efc39
Found cached dataset csv (/home/shahul/.cache/huggingface/datasets/merve___csv/merve--poetry-f27664155e5efc39/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [43]:
poem_df = pd.read_csv("/home/shahul/Data/PoetryFoundationData.csv")

In [44]:
author_dict = poem_df['Poet'].value_counts().to_dict()
poem_df["poems_counts"] = poem_df["Poet"].map(author_dict)

In [45]:
poem_df = poem_df[poem_df["poems_counts"]>10]
poem_df = poem_df.rename({"Poet":"author","Poem":"content",
                "Title":'poem name',"Tags":"type"},axis=1)
poem_df = poem_df[['poem name', 'content', 'author', 'type']]
poem_df["age"] = ""
poems_dataset = Dataset.from_pandas(poem_df,split="train")

(5581, 6)

### Templates

In [ ]:
poem_synonyms = ["poem","poetry"]
compose_synonyms = ["Write","Help me write", "Compose", "Please craft", "Give me"]
complete = ["Complete", "Finish", "Put the finishing touches to"]


In [ ]:
prompts =
[f"{Compose} a {poem} entitled {title} in the {author}'s writing style.",
 f"{Compose} a {poem} on {title} in the manner of {author}.", ## if title startswith "The"
 f"{Compose} a {poem} about {title} in the manner of {author}.", ## if title startswith "The"
 f"{Compose} a {poem} containing {rhyming_words}.",
 f"{Compose} a {poem} containing {rhyming_words} in {author}'s writing style.",
 f"{Compose} a {poem} with the type {genre} written in {age}.", 
 f"{Compose} a {poem} of the genre {genre} during the {age}.
 ## complete poem
 f"{complete} the {poem}",
 f"{complete} the {poem} in {author}'s writing style",
 f"{complete} the {poem} entitled {title}",
 
 
]

## Augmentation